In [1]:
import numpy as np 
import pandas as pd

def weighted_frequency(recommendations):
    weights = {1: -1.5, 2: -1, 3: 0, 4: 1, 5: 1.5}
    weighted_recommendations = recommendations.groupby('Symbol').apply(
        lambda x: sum(x['Call'].apply(lambda y: weights[y]))
    ).reset_index(name='Weighted_Sum_Call')
    
    # Add the date column
    weighted_recommendations['Date'] = recommendations.name
    return weighted_recommendations


In [2]:
from glob import glob

# read MadMoneyInversedData_no_lightning_*.csv and concatenate them into one dataframe
filenames = glob('MadMoneyInversedData_no_lightning_*.csv')
recommendations_df = pd.concat([pd.read_csv(f, encoding='ISO-8859-1') for f in filenames])

In [3]:
recommendations_df['Call'].value_counts()

Call
1    7667
2    1649
4    1355
5     473
3       7
Name: count, dtype: int64

In [4]:
# Convert the date column to datetime format
recommendations_df['Date'] = pd.to_datetime(recommendations_df['Date'])

# Set the date as the index
# recommendations_df.set_index('Date', inplace=True)

# Resample the recommendations to a monthly frequency and apply the weighted frequency function
monthly_recommendations = recommendations_df.resample('M', on='Date').apply(weighted_frequency).set_index('Date')


In [5]:
# get row '2017-01-31' from monthly_recommendations
monthly_recommendations.loc['2017-01-31'].sort_values(by='Weighted_Sum_Call', ascending=False)

,Symbol,Weighted_Sum_Call
Date,,
2017-01-31,M,3.5
2017-01-31,TGT,3.0
2017-01-31,UA,2.5
2017-01-31,KSS,2.5
2017-01-31,FSLR,2.5
...,...,...
2017-01-31,AMZN,-6.5
2017-01-31,BAC,-7.0
2017-01-31,JPM,-8.5


In [6]:
monthly_recommendations = monthly_recommendations.sort_values('Weighted_Sum_Call', ascending=False)
grouped = monthly_recommendations.groupby('Date').apply(lambda x: pd.Series({'Top5': x.head(5)['Symbol'].tolist(), 'Bottom5': x.tail(5)['Symbol'].tolist()}))

print(grouped)

                                     Top5                         Bottom5
Date                                                                     
2017-01-31        [M, TGT, KSS, UA, FSLR]     [AMZN, BAC, JPM, AAPL, UNH]
2017-02-28       [TWTR, KSS, M, NWL, TWX]    [HAS, FB, GOOGL, AMZN, AAPL]
2017-03-31  [SNAP, SHLD, SPG, TRIP, TWNK]     [DRI, DPZ, ADBE, RHT, AAPL]
2017-04-30       [SBUX, S, WFM, SAM, TAP]   [CAT, LRCX, MCD, AMZN, GOOGL]
2017-05-31      [WFM, TGT, SYF, AXP, AKS]  [CRM, TTWO, AMZN, GOOGL, AAPL]
2017-06-30     [BBBY, KR, NKE, SVU, RCII]  [GOOGL, STZ, AAPL, NVDA, ADBE]
2017-07-31      [APRN, GE, CMG, AAP, AZO]    [CAT, NVDA, FB, GOOGL, NFLX]
2017-08-31      [LB, INSY, SNAP, FL, FIT]      [PVH, NVDA, V, AMZN, AAPL]
2017-09-30   [BBBY, EFX, FINL, COTY, CVS]   [NFLX, BURL, TJX, AMZN, AAPL]
2017-10-31       [GE, KHC, WBA, NKE, CMG]       [SWK, GM, JPM, CAT, AAPL]
2017-11-30     [JCP, GE, CVS, SNAP, SFIX]     [HD, TSN, PYPL, NVDA, AAPL]
2017-12-31   [ADSK, TVPT, SJM, PLAY, P

In [7]:
# save group to csv
grouped.to_csv('cramer_top_picks_2017-2022.csv')

In [41]:
# print all unique symbols in grouped (top5 and bottom5)
symbols = set()
for top5, bottom5 in zip(grouped['Top5'], grouped['Bottom5']):
    symbols.update(top5)
    symbols.update(bottom5)
print(len(symbols))

258


In [8]:
line = """
2021-12-31,"['BZFD', 'PRG', 'BB', 'BIRD', 'W']","['DIS', 'WSM', 'AAPL', 'LLY', 'F']"
"""

import re
date_str, top5_str, bottom5_str = re.split(',(?![^[]*\])', line)
top5_str

'"[\'BZFD\', \'PRG\', \'BB\', \'BIRD\', \'W\']"'

In [18]:
eval(eval(bottom5_str))[0]

'DIS'

In [10]:
import ast

e = ast.literal_eval(top5_str)

In [12]:
eval(e)

['BZFD', 'PRG', 'BB', 'BIRD', 'W']

list